# Importing packages

In [2]:
from workers import MasterNode
from models import LinReg, LogReg, LogRegNoncvx, NN_1d_regression
from utils import read_run, get_alg, create_plot_dir, PLOT_PATH
from sklearn.datasets import dump_svmlight_file

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from prep_data import number_of_features
import math
import torch

from numpy.random import default_rng
from numpy import linalg as la
from prep_data import DATASET_PATH
import copy
import pickle
import sys
from multiprocessing import Process

# Customizing Matplotlib

In [3]:
plt.style.use('fast')
mpl.rcParams['mathtext.fontset'] = 'cm'
# mpl.rcParams['mathtext.fontset'] = 'dejavusans'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['lines.linewidth'] = 2.0
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

In [4]:
markers = ['x', '.', '+', '1', 'p','*', 'D' , '.',  's']

In [5]:
MODELS_PATH = 'models/'

# Generating datasets

In [ ]:
def create_datasets(number_of_tasks,
                    n1, 
                    number_of_points_per_task, 
                    number_of_points_per_task_for_validation, 
                    dataset_name,
                    validation_dataset_name,
                    keyword
                   ):    
    X_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task, 1))
    y_sine = np.empty(shape=(number_of_tasks * number_of_points_per_task))
    X_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation, 1))
    y_sine_val = np.empty(shape=(number_of_tasks * number_of_points_per_task_for_validation))
    
    n2 = number_of_tasks - n1
    
    rng = default_rng()
    a_array = []
    b_array = []
    a_array_old = np.load(DATASET_PATH + 'a_two_distribution_validated.npy')
    b_array_old = np.load(DATASET_PATH + 'b_two_distribution_validated.npy')
    dist_sizes = [n1, n2]
    
    for j in range(2):
        a = a_array_old[100 * j]
        b = b_array_old[100 * j]
        for i in range(dist_sizes[j]):
            a_array.append(a)
            b_array.append(b)

            x_train = -5.0 + rng.random((number_of_points_per_task, 1)) * 10.0
            x_val = -5.0 + rng.random((number_of_points_per_task_for_validation, 1)) * 10.0
            y_train = a * np.sin(x_train + b)
            y_val = a * np.sin(x_val + b)

            ind = i + dist_sizes[0] * j
            X_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = x_train.copy()
            X_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = x_val.copy()    
            y_sine[number_of_points_per_task * ind : number_of_points_per_task * (ind + 1)] = y_train.squeeze(1).copy()
            y_sine_val[number_of_points_per_task_for_validation * ind : number_of_points_per_task_for_validation * (ind + 1)] = y_val.squeeze(1).copy()    
    a_array = np.array(a_array)
    b_array = np.array(b_array)
    dump_svmlight_file(X_sine, y_sine, DATASET_PATH + dataset_name)
    dump_svmlight_file(X_sine_val, y_sine_val, DATASET_PATH + validation_dataset_name)
    np.save(DATASET_PATH + 'a' + keyword + '.npy', a_array)
    np.save(DATASET_PATH + 'b' + keyword + '.npy', b_array)

# Optimization

In [ ]:
def save(models, filename='a_constant_1000'):
    models_dict = {'models' : models}
    with open(MODELS_PATH + filename, 'wb') as file:
        pickle.dump(models_dict, file)

In [ ]:
def global_model(number_of_tasks, 
                 dataset_name, 
                 validation_dataset_name,
                 keyword,
                 model
                ):
    models = []
    for i in range(11):
        models.append(copy.deepcopy(model))
    alphas = np.linspace(0, 1.0, 11)
    for i in range(1,11):
        models[i].change_alpha(alphas[i])
    models[0].change_alpha(0.0)
    models[0].w_opt_global = np.zeros(models[0].d)
    saves_name = 'two_distribution' + keyword
    for i in range(1, 11):
        model = models[i]
        min_L = 0.1
        max_L = 0.1
        max_it = 10000
        tol = 1e-2
        max_L_constant = 2 ** 40
        w = copy.deepcopy(models[i-1].w_opt_global)
        grad_norm = None
        min_f_value = float('Inf')

        try:
            for it in range(max_it):
                grad = model.grad(w)
                L = min_L
                curr_fun_value = model.fun_value(w)

                while True:
                    if L > max_L_constant: # if L becomes too large, jump to another random point w
                        w = np.random.randn(model.d)
                        grad = model.grad(w)
                        L = min_L
                        curr_fun_value = model.fun_value(w)

                    print('Current L = {:f}'.format(L), end='\r')

                    f_value_ = model.fun_value(w - grad / L)
                    if curr_fun_value - f_value_ > 0:
                        break
                    L *= 2.0

                w -= grad / L
                grad_norm = la.norm(grad)

                if f_value_ is None:
                    raise Exception('None is detected') 

                if f_value_ < min_f_value:
                    min_f_value = f_value_
                    model.w_opt_global = copy.deepcopy(w)

                if max_L < L:
                    max_L = L
                print('                               {:5d}/{:5d} Iterations: fun_value {:f} grad_norm {:f}'.format(it+1, max_it, f_value_, grad_norm), end='\r')                
                if grad_norm < tol and f_value_ < tol ** 2:
                    save(models, saves_name)
                    break
        except KeyboardInterrupt:
            print('')
            print(min_f_value)
        else:
            save(models, saves_name)
            print('')

In [ ]:
def two_distribution(number_of_tasks, first_dataset_size, validation=True):
    n1 = first_dataset_size
    n2 = number_of_tasks - n1
    keyword = str(n1) + 'vs' + str(n2)
    if validation:
        keyword += 'with_validation_stop'
    dataset_name = 'artificial_sine_two_distribution' + keyword
    validation_dataset_name = 'artificial_sine_two_distribution_validation' + keyword
    
    file_out = open(keyword + '.txt', 'w')
    sys.stdout = file_out
       
    model = MasterNode(n_workers=number_of_tasks, 
                   alpha=0.05, 
                   worker=NN_1d_regression, 
                   dataset_name=dataset_name, 
                   logreg=False, 
                   ordered=True, 
                   max_it=100, 
                   tolerance=1e-2, 
                   validation=validation, 
                   validation_dataset_name=validation_dataset_name)
  
    global_model(number_of_tasks, dataset_name, validation_dataset_name,keyword, model)
    file_out.close()

In [ ]:
# # creating datasets
# for i in np.arange(10, 110, 10):
#         number_of_tasks = 200
#     n1 = i
#     n2 = number_of_tasks - n1
#     number_of_points_per_task = 50
#     number_of_points_per_task_for_validation = 20
#     keyword = str(n1) + 'vs' + str(n2)
#     dataset_name = 'artificial_sine_two_distribution' + keyword
#     validation_dataset_name = 'artificial_sine_two_distribution_validation' + keyword
    
#     create_datasets(number_of_tasks,
#                 n1, 
#                 number_of_points_per_task, 
#                 number_of_points_per_task_for_validation, 
#                 dataset_name,
#                 validation_dataset_name,
#                 keyword
#                )

# Experiments

In [ ]:
sizes = np.arange(10, 110, 10)

In [ ]:
# validation stop critetion experiments
processes = []
ind = 0
for first_dataset_size in sizes:
    processes.append(Process(target=two_distribution, args=(200, first_dataset_size, True)))
    processes[ind].start()
    ind += 1
for ind in range(sizes.size):
    processes[ind].join()

In [ ]:
# funcvalue stop criterion
processes = []
ind = 0
for first_dataset_size in sizes:
    processes.append(Process(target=two_distribution, args=(200, first_dataset_size, False)))
    processes[ind].start()
    ind += 1
for ind in range(sizes.size):
    processes[ind].join()

# Plots

In [ ]:
validation = True
if validation:
    print('Drawing graphs for mixed models with validation stop criterion')    
else:
    print('Drawing graphs for mixed models with functional value stop criterion')
    
for first_dataset_size in sizes:
    keyword = str(first_dataset_size) + 'vs' + str(200 - first_dataset_size)
    
    a_array = np.load(DATASET_PATH + 'a' + keyword + '.npy')
    b_array = np.load(DATASET_PATH + 'b' + keyword + '.npy')
    if validation:
        keyword += 'with_validation_stop'
    model_name = 'two_distribution' + keyword
    with open(MODELS_PATH + model_name , 'rb') as file:
        models = pickle.load(file)['models']
        
    mse_table = np.empty(shape=(11, models[0].n_workers))
    
    n_test = 2000
    rng = default_rng()
    for i in range(models[0].n_workers):
        x_test = -5.0 + rng.random((n_test, 1)) * 10
        y_test = a_array[i] * np.sin(x_test + b_array[i])
        for j in range(11):
            worker = models[j].workers[i]
            worker.set_weights(worker.compute_local(models[j].w_opt_global))
            y_pred = worker.model(torch.from_numpy(x_test).float()).detach().numpy()
            mse = np.mean((y_pred - y_test) ** 2).item()
            mse_table[j][i] = copy.copy(mse)
            
    mse_alpha = np.mean(mse_table, axis=1)
    alphas = np.linspace(0, 1.0, 11)
    argmin = np.argmin(mse_alpha)
    alpha_min = alphas[argmin]
    plt.figure()
    plt.plot(alphas, mse_alpha, marker='o')
    plt.yscale('log')
    plt.xlabel('alpha')
    plt.ylabel('Average MSE over clients')
    plt.xticks(alphas)
    plt.axvline(x=alpha_min, ymin = 0, ymax=1, ls='--')
    plt.title(keyword)
    plt.tight_layout()
    plt.savefig(PLOT_PATH + '/' + keyword + '.pdf')
    plt.figure()
    mse_best_log = np.log(mse_table[argmin])
    plt.hist(mse_best_log, bins=50)
    plt.xlabel('Log MSE for the best alpha')
    plt.title(keyword)
    plt.savefig(PLOT_PATH + '/hist' + keyword + '.pdf')